In [7]:
import sqlite3
import pandas as pd
from datetime import datetime
import creditcard
from sklearn.metrics.pairwise import cosine_similarity

from sentence_transformers import SentenceTransformer
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import re
import creditcard
import numpy as np

[nltk_data] Downloading package punkt to /home/zq/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
pdf_path = r"/home/zq/Downloads/Citibank_CreditCard_7099stmt_19082022_1680435958485.pdf"
df = creditcard.read_creditcard(pdf_path)

/home/zq/Desktop/code/.conda/lib/python3.10/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 1
  warnings.warn(f"No tables found in table area {table_idx + 1}")
/home/zq/Desktop/code/simple_budget_streamlit/creditcard.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append(df)


In [3]:
df

2,DATE,DESCRIPTION,AMOUNT (SGD)
7,17 JUL,NTUC FairPrice App Pay,111.55
8,19 JUL,NTUC FairPrice App Pay,2.80
9,20 JUL,NTUC FairPrice App Pay,5.40
10,25 JUL,GV ONLINE,-29.00
11,25 JUL,Grab* 1658731794000000,100.00
12,25 JUL,OG ORCHARD POINT,383.81
13,30 JUL,Spotify P1CD88ABE6\nStockholm\nSE,9.90
14,30 JUL,SAND&SANDALS RESORT &\nKOTA TINGGI MY,305.87
16,31 JUL,DESARUCOAST-ADVENTUREP\nBDR PENAWAR MY,113.94
18,31 JUL,DESARUCOAST-ADVENTUREP\nBDR PENAWAR MY,8.05


In [4]:
def clean_text_BERT(text):

    # Convert words to lower case.
    text = text.lower()

    # Remove special characters and numbers. This also removes the dates 
    # which are not important in classifying expenses
    text = re.sub(r'[^\w\s]|https?://\S+|www\.\S+|https?:/\S+|[^\x00-\x7F]+|\d+', '', str(text).strip())
  
    # Tokenise 
    text_list = word_tokenize(text)
    result = ' '.join(text_list)
    return result

In [5]:
text_raw = df['DESCRIPTION']
text_BERT = text_raw.apply(lambda x: clean_text_BERT(x))

In [8]:
bert_input = text_BERT.tolist()
model = SentenceTransformer('paraphrase-mpnet-base-v2') 
embeddings = model.encode(bert_input, show_progress_bar = True)
embedding_BERT = np.array(embeddings)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
embedding_BERT.shape

(19, 768)

In [6]:
# from transformers import BertTokenizer, BertModel
# tokenizer = BertTokenizer.from_pretrained('zanelim/singbert')
# model = BertModel.from_pretrained("zanelim/singbert")

# encoded_input = tokenizer(text, return_tensors='pt')
# text_output = model(**encoded_input)

# encoded_input = tokenizer(label, return_tensors='pt')
# label_output = model(**encoded_input)

### Idea 1: Compare desc directly to categories
Don't think this will work well, span of text is too different

### Idea 2: Group those that is most similar, then perform batch labelling
Requires data, need to search for public datasources

In [12]:
categories = ['Rent', 'Utilities', 'Travel', 'Electronics', 'Gym', 'Telco']